# Access data using `enty`

This tutorial walks you through how to access the exported data. The data was exported using the in-house Python package `enty` which mirrors aspects of `datajoint` except that it works with folders and filepaths rather than a database.

# Import `enty` schemas

The `enty` schema code is located in `mouse_neuropixel_export.schemas`. Conceptually, `enty` schemas are equivalent to `datajoint` schemas in that they are a collection of data tables. Each `enty` schema maps to a directory, and the tables within the schema map to subdirectories. 

In [1]:
from mouse_neuropixel_export.schemas import metadata, experiment

# Set the schema directory

In order for the schema to access that data, the schema object must point to the schema directory path. 

Inside the exported directory `mouse_neuropixel_export` should be a directory called `schemas`.

Update the `schema_dir` variable in the cell below to the correct path for your system.

In [2]:
schema_dir = '/mnt/at-export01/mouse_neuropixel_export/schemas/'

metadata.schema.base_dir = schema_dir + 'metadata'
experiment.base_dir = schema_dir + 'experiment'

# Access schema tables

The tables within a schema can be accessed from the schema object. Each table corresponds to a subdirectory inside the schema folder. 

For example in the `metadata` directory there exists a subdirectory `.../schemas/metadata/scan_key`. The corresponding table in the schema is `metadata.ScanKey`.

This can be accessed as shown in the cell below. When accessed with no arguments, the table returns a `pandas` dataframe view of the underlying data in the subdirectory, where each row of the dataframe is one file in the subdirectory.

The columns of the table represents the key that is used to identify each file, equivalent to primary keys in `datajoint`.

In [3]:
metadata.ScanKey()

,animal_id,session,scan_idx
0,27152,17,1
1,27152,26,1
2,27152,28,1
3,27152,31,1
4,28470,13,1
5,28488,10,1
6,28488,16,1
7,28489,16,1
8,28489,20,1
9,28489,21,1


# Access the filepath to the data in the table

To instantiate the file path object for any given file in the table, pass the key to the table object.

In [4]:
key = {'animal_id': 27152, 'session': 17, 'scan_idx': 1}
fp_object = metadata.ScanKey(**key)
fp_object

ScanKey object with key: ScanKey.Key(animal_id=27152, session=17, scan_idx=1)

The filepath can then be accessed with the `get_path` method:

In [5]:
fp_object.get_path()

PosixPath('/mnt/at-export01/mouse_neuropixel_export/schemas/metadata/scan_key/metadata_scan_key____27152__17__1')

The returned path is agnostic to file type extension. If you know the extension you can pass it in as follows:

In [6]:
fp_object.get_path('.pkl')

PosixPath('/mnt/at-export01/mouse_neuropixel_export/schemas/metadata/scan_key/metadata_scan_key____27152__17__1.pkl')

# Access the data for a filepath

Once the filepath is accessed, the data can be loaded using the filepath with whatever method desired. 

However in this case, each table already has a `get` method implemented that takes care of the mapping to the data.

In the cell below we call the `get()` function in order to get the data from the file.

In this case, the data that is returned is just a dictionary that is the same as the key because this is a metadata schema. 

However, in other cases, the data that is returned can be of any type. 

In [7]:
data = fp_object.get()
data

{'animal_id': 27152, 'session': 17, 'scan_idx': 1}

Below, we access the data from the `experiment.Session` table using the same key. 

Now the data is a pandas dataframe with the session information for this animal. 

In [8]:
data = experiment.Session(**key).get()
data

,animal_id,session,rig,session_date,username,anesthesia,scan_path,behavior_path,craniotomy_notes,session_notes,archive,session_ts
0,27152,17,RS1,2022-03-21,maria,awake,N:\Two-Photon\maria\2022-03-21_14-45-34,N:\Two-Photon\maria\2022-03-21_14-45-34,,,RS1A1A,2022-03-21 12:45:34


This is all the functionality that is necessary to get started, and the subsequent tutorials will give more specific instructions for accessing and using the data.